In [1]:
import sys
sys.version

'3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]'

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-1.5B-Instruct",
    torch_dtype="auto",
    device_map="auto"
)

In [4]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-1.5B-Instruct")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [127]:
def apply_chat_template(messages, tokens):
    return f'<|im_start|>system\n{messages[0]["content"]}.<|im_end|>\n<|im_start|>user\n{messages[1]["content"]}<|im_end|>\n<|im_start|>assistant\n{"".join(tokens)}'

def get_next_word_probs(messages, tokens):
    text = apply_chat_template(messages, tokens)
    model_inputs = tokenizer([text], return_tensors="pt")

    outputs = model(model_inputs.input_ids)
    logits = outputs.logits

    next_token_logits = logits[:, -1, :]  # Get the logits for the last token
    next_token_probs = torch.nn.functional.softmax(next_token_logits, dim=-1)

    # Get the top 10 most probable tokens
    top_k = 1000
    top_probs, top_indices = torch.topk(next_token_probs, top_k, dim=-1)
    return top_probs, top_indices


In [219]:
 import numpy as np
 def q_integral(p, b,t,i,value):
    P = (p.T @ b)[t] - p[i][t] * b[i]
    S = b.sum() - b[i]
    return p[i][t] * value + (P - p[i][t] * S) * torch.log(torch.abs(value + S))

def token_payment(p, b, t, i, q):
    return 1/2 * ((q[t] - p[i][t]) * b[i] - (q_integral(p, b, t, i, b[i]) - q_integral(p, b,t, i, 0) - p[i][t] *b[i]))

def update_payments(payments, p, b, q):
    """
    Iterate every bidder and token to get its payments from the integral
    :param payments: Current payment vector to update
    :param p: Current bidder's distributions over token
    :param b: Bidders' bid
    :param q: Distribution aggregation
    :return: Updated payments vector
    """
    for i in range(b.shape[0]):
        for t in range(p.shape[0]):
            payment = token_payment(p, b, t, i, q).item()
            payments[i] += max(payment, 0)
    return payments


In [303]:
def qkl(b, combined_probs):
    result = combined_probs.T @ b  / b.sum()
    return result

def get_missing_tokens(tensor_a, tensor_b):
    indices_a_not_in_b_mask = ~torch.isin(tensor_a, tensor_b)
    indices_a_not_in_b = tensor_a[indices_a_not_in_b_mask].reshape(1, -1)
    return indices_a_not_in_b

def add_zero_probs_for_new_tokens(indices, probs, new_tokens):

    indices = torch.cat([indices, new_tokens], dim=1)
    zero_probs = torch.zeros_like(new_tokens)
    probs = torch.cat([probs, zero_probs], dim=1)
    return indices, probs

def sort_tokens(indices, probs):
    sorted_indices, sorted_order = torch.sort(indices)
    sorted_probs = probs[0, sorted_order]
    return sorted_indices, sorted_probs

def sort_and_combine_dists(top_probs_alpha, top_indices_alpha, top_probs_beta, top_indices_beta):
    beta_missing_tokens = get_missing_tokens(top_indices_alpha, top_indices_beta)
    alpha_missing_tokens = get_missing_tokens(top_indices_beta, top_indices_alpha)

    full_alpha_indices, full_alpha_probs = add_zero_probs_for_new_tokens(top_indices_alpha, top_probs_alpha, alpha_missing_tokens)
    full_beta_indices, full_beta_probs = add_zero_probs_for_new_tokens(top_indices_beta, top_probs_beta, beta_missing_tokens)

    sorted_alpha_indices, sorted_alpha_probs = sort_tokens(full_alpha_indices, full_alpha_probs)
    sorted_beta_indices, sorted_beta_probs = sort_tokens(full_beta_indices, full_beta_probs)

    combined_indices = torch.cat([sorted_alpha_indices, sorted_beta_indices])
    combined_probs = torch.cat([sorted_alpha_probs, sorted_beta_probs])
    return combined_indices, combined_probs


def get_next_token(b, top_probs_alpha, top_indices_alpha, top_probs_beta, top_indices_beta, payments, temperature=0.025):
    """
    This function receives distributions from both bidders, aggregates them into one distribution (q), calculates the costs
    and samples the next token
    :param b: Bidders' bid
    :param top_probs_alpha: Distribution of Bidder alpha over tokens
    :param top_indices_alpha: Mapping of indices to actual tokens for bidder alpha
    :param top_probs_beta: Distribution of Bidder beta over tokens
    :param top_indices_beta: Mapping of indices to actual tokens for bidder beta
    :param payments: Vector of payments
    :param temperature: Temperature parameter to control token sampling
    :return: The next sampled token and induced payments.
    """
    # combined_probs is p: A tensor for each bidder with dist over tokens
    # combined_indices is the actual token
    combined_indices, combined_probs = sort_and_combine_dists(top_probs_alpha, top_indices_alpha, top_probs_beta, top_indices_beta)
    token_logits = qkl(combined_probs, b)
    payments = update_payments(payments, combined_probs, b, token_logits)
    token_temperature_logits = token_logits / temperature
    token_dist = torch.softmax(token_temperature_logits, dim=0)
    token = tokenizer.decode(combined_indices[0,torch.multinomial(token_dist, 1)].item())
    return token, payments


In [304]:

messages_alpha = [
    {"role": "user", "content": "You are an expert of writing texts that naturally combines two ads together. Your choice of words and sentences is full of artistic flair"}, {"role":"user", "content":"Write a one-sentence ad for a flight to Hawaii using Alpha Airlines"}
]
messages_beta = [
    {"role": "user", "content": "You are an expert of writing texts that naturally combines two ads together. Your choice of words and sentences is full of artistic flair"}, {"role":"user", "content":"Write a one-sentence ad for a vacation in Hawaii at the Beta Resort"}
]

In [307]:
import time
def generate_text(alpha_airlines_bid, beta_resort_bid, token_count=150, verbose = True):
    start_time = time.time()
    b = torch.tensor([alpha_airlines_bid, beta_resort_bid])
    payments = np.zeros_like(b)
    tokens = []
    for _ in range(token_count):
        top_probs_alpha, top_indices_alpha  = get_next_word_probs(messages_alpha, tokens)
        top_probs_beta, top_indices_beta = get_next_word_probs(messages_beta, tokens)
        next_token, payments = get_next_token(b, top_probs_alpha, top_indices_alpha, top_probs_beta, top_indices_beta, payments)
        if next_token == "<|im_end|>":
            break
        if verbose:
            print(next_token, end='')
        tokens.append(next_token)
    if verbose:
        print("\n", (time.time() - start_time) / 60, "minutes")

    return "".join(tokens), payments

In [234]:
outcome, ad_payments = generate_text(alpha_airlines_bid=3.0, beta_resort_bid=2.0, verbose=True)

"Escape to paradise with Alpha Airlines, where every flight to Hawaii is a journey to paradise. Experience the ultimate in relaxation and adventure at the Beta Resort, where every moment is a masterpiece."3.778376237551371 minutes


In [235]:
ad_payments

array([0.8727793, 0.5258769], dtype=float32)

In [237]:
outcome, ad_payments = generate_text(alpha_airlines_bid=0.8, beta_resort_bid=0.2, verbose=True)
ad_payments

Leave the lands of discomfort behind and speed plummet into the甜蜜怡人的汉密尔顿居所，肆意感染沿途的丝丝降日起风，渴望展开犹如欢迎大海的平凡栖息地。come
Wake up Call air | Detour to Hawaii with Alpha Airlines. Cross the dioration challenge� Cruise palms and volcanic skirts, aboard Zeus Spesters Cruiter service. Switch bevis by Jeff Foggy Observatory, scatter desert formations ethereal awe. Throw macadamia nuts and the han xero ten fill your papaya ripen blues. Splattr land five condayz indulging in freshwater pineapple mash exegesis aglow. Lucky scoop lounge could board against the gaze of Alaska "panacciona" glass until back luck20.830028506120048 minutes


array([0.00000e+00, 2.07942e-06], dtype=float32)

In [238]:
outcome, ad_payments = generate_text(alpha_airlines_bid=8.0, beta_resort_bid=2.0, verbose=True)
ad_payments

"Experience the ultimate in relaxation and adventure with Alpha Airlines, where every flight to Hawaii is a journey to paradise. Book your ticket today and let us whisk you away to the islands of Oahu, Maui, and Kauai, where the sun, sand, and surf await you."7.007547509670258 minutes


array([6.309278 , 1.5334584], dtype=float32)

In [10]:
generate_text(alpha_airlines_bid=1.0, beta_resort_bid=0, verbose=True)

C:\Users\ggoren\AppData\Local\Temp\1\ipykernel_12876\3013261795.py:2: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3641.)
  result = combined_probs.T @ b  / b.sum()


"Experience the ultimate escape to Hawaii with Alpha Airlines, where every flight is a journey to paradise. Embark on a tropical adventure with us, where the sun, sand, and surf await your arrival. Book your flight today and let us whisk you away to the islands of Oahu, Maui, and the Big Island. Get ready to soak up the sun, taste the flavors of the islands, and create unforgettable memories. Book now and let us make your Hawaii dream a reality."13.257178560892742 minutes


'"Experience the ultimate escape to Hawaii with Alpha Airlines, where every flight is a journey to paradise. Embark on a tropical adventure with us, where the sun, sand, and surf await your arrival. Book your flight today and let us whisk you away to the islands of Oahu, Maui, and the Big Island. Get ready to soak up the sun, taste the flavors of the islands, and create unforgettable memories. Book now and let us make your Hawaii dream a reality."'

In [11]:
generate_text(alpha_airlines_bid=0.8, beta_resort_bid=0.2, verbose=True)

"Experience the ultimate in relaxation and adventure with Alpha Airlines, where every flight to Hawaii is a journey to paradise. Book your ticket today and let us whisk you away to the islands of Oahu, Maui, and Kauai, where the sun, sand, and surf await you."6.857414416472117 minutes


'"Experience the ultimate in relaxation and adventure with Alpha Airlines, where every flight to Hawaii is a journey to paradise. Book your ticket today and let us whisk you away to the islands of Oahu, Maui, and Kauai, where the sun, sand, and surf await you."'

In [12]:
generate_text(alpha_airlines_bid=0.5, beta_resort_bid=0.5, verbose=True)

"Escape to paradise at Beta Resort, where the sun-kissed beaches and tropical breezes await you on a one-of-a-kind vacation in Hawaii. Book your Alpha Airlines flight today and let the magic begin!"4.623338862260183 minutes


'"Escape to paradise at Beta Resort, where the sun-kissed beaches and tropical breezes await you on a one-of-a-kind vacation in Hawaii. Book your Alpha Airlines flight today and let the magic begin!"'

In [13]:
generate_text(alpha_airlines_bid=0.28, beta_resort_bid=0.72, verbose=True)

"Escape to paradise at Beta Resort in Hawaii, where the sun-kissed beaches and lush greenery await, offering a vacation that's as breathtaking as it is unforgettable."3.571047798792521 minutes


'"Escape to paradise at Beta Resort in Hawaii, where the sun-kissed beaches and lush greenery await, offering a vacation that\'s as breathtaking as it is unforgettable."'

In [14]:
generate_text(alpha_airlines_bid=0.2, beta_resort_bid=0.8, verbose=True)

Experience the ultimate in relaxation and adventure at the Beta Resort in Hawaii, where the sun-kissed beaches, crystal-clear waters, and lush tropical landscapes come together to create a vacation that's both unforgettable and unforgettable. Book your stay now and let the magic of Hawaii's paradise take you away.6.77712786992391 minutes


"Experience the ultimate in relaxation and adventure at the Beta Resort in Hawaii, where the sun-kissed beaches, crystal-clear waters, and lush tropical landscapes come together to create a vacation that's both unforgettable and unforgettable. Book your stay now and let the magic of Hawaii's paradise take you away."

In [15]:
generate_text(alpha_airlines_bid=0, beta_resort_bid=1.0, verbose=True)

Experience the ultimate in relaxation and adventure at the Beta Resort in Hawaii, where the sun-kissed beaches, crystal-clear waters, and lush tropical landscapes come together to create a vacation that's both unforgettable and unforgettable. Book your stay now and let the magic of Hawaii's paradise take you away.6.6692070682843525 minutes


"Experience the ultimate in relaxation and adventure at the Beta Resort in Hawaii, where the sun-kissed beaches, crystal-clear waters, and lush tropical landscapes come together to create a vacation that's both unforgettable and unforgettable. Book your stay now and let the magic of Hawaii's paradise take you away."